# Access4All — 01_airbnb_data_loader

This notebook loads the raw Airbnb listings dataset and performs basic cleaning.
The processing here is limited to removing irrelevant columns and preparing a clean base dataset for downstream layers.


In [0]:

from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql import functions as F

### Configure secure access to the Azure Blob Storage container


In [0]:
storage_account = "lab94290"  
container = "airbnb"

sas_token = "<Insert sas token here>"
sas_token = sas_token.lstrip('?')
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "SAS")
spark.conf.set(f"fs.azure.sas.token.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set(f"fs.azure.sas.fixed.token.{storage_account}.dfs.core.windows.net", sas_token)

### Load the raw Airbnb dataset from cloud storage


In [0]:
path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/airbnb_1_12_parquet"

air_bnb_data = spark.read.parquet(path)


name price image description category availability discount reviews ratings seller_info breadcrumbs location lat long guests pets_allowed description_items category_rating house_rules details highlights arrangement_details amenities images available_dates url final_url listing_title property_id listing_name location_details description_by_sections description_html location_details_html is_supperhost host_number_of_reviews host_rating hosts_year host_response_rate is_guest_favorite travel_details pricing_details total_price currency cancellation_policy property_number_of_reviews country postcode_map_url host_image host_details Rental unit in Broadbeach · ★4.96 · 2 bedrooms · 2 beds · 1 bath 238 https://a0.muscache.com/pictures/50ddeaed-5302-4d0c-8200-163b2cddd071.jpg Our exceptionally stylish two bedroom, one bathroom LUXURY holiday apartment is located in the ORACLE T2, right in the heart of beautiful Broadbeach! Every corner of this gorgeous apartment overflows with style, luxury and thoughtful touches. Relax on the ocean facing balcony located directly above the manicured garden terrace and take in the relaxing views and sea breezes. This property is all class and guaranteed to satisfy even the fussiest Airbnb guest!The spaceThe photos speak for themselves. They are a true, accurate and up to date representation of our immaculately presented, squeaky clean, apartment. The furnishings are practical and comfortable and the linen and bedding are soft and luxurious. The master king bedroom opens out onto the large balcony. The second bedroom is a small internal room with a comfortable high quality queen sized bed. The second bedroom does not have a window. Each bedroom has an internal sliding door which provides access to the two way bathroom. Our apartment has been styled with a relaxing holiday in mind.Guest accessPoolSpaGymSauna and steam roomBBQ and entertaining areasGardensSecure off street parking for ONE vehicleAccess to facilities is not guaranteed and may be interrupted by routine and unexpected maintenance schedules.Other things to noteThis property accommodates up to FOUR people only. Children aged two and older are included in the four people rule. No exceptions. Infants UNDER the age of two are very welcome at our place and may stay at no additional charge however we do not provide linen or bedding. Let us know if you would like the link to a local business who hires out baby equipment (travel cots, strollers, high chairs).CHECK IN Available between 3pm and 9pm.Self check in is NOT available at this property. After hours check ins are NOT available. A 3pm check in is not guaranteed and depends on cleaning schedules. CHECK IN APPOINTMENTOur lovely host welcomes you in person on arrival. The booking guest is required to make contact with us via the booking message thread prior to their arrival day to arrange a check in time. A host will not be onsite to meet you unless you have scheduled a check in time. We always do our best to fit in with your travel schedule. IMPORTANTKeys and security fobs will only be issued to the person who made the booking. ID will be required. This is not negotiable. Please read the HOUSE RULES and CANCELLATION POLICY prior to booking our place. We recommend taking out travel insurance to cover any losses which may occur if you need to cancel your booking for an unforeseen reason. NO SMOKING - smoking is NOT permitted anywhere at this complex.CHECK IN / CHECK OUT - NOT AVAILABLE* Christmas Day* New Years Day * Easter SundayEARLY CHECK IN - We will always accommodate an early check in if our place is available. CHECK OUT by 11am - Late check outs are not available and hold up our cleaning team. WHAT WE INCLUDE IN OUR STARTER PACKOur starter pack is designed to help get you started on your holiday and may not last the entire trip. If you are with us for more than two nights, you may need to re stock items at the local supermarket. Woolworths supermarket is located right next door for any additi

### Select only the core columns required for downstream processing


In [0]:
v1_columns = [
    "amenities",
    "details",
    "lat",
    "long",
    "location",
    "country",
    "property_id",
    "url"
]




### Remove rows missing essential identifying or descriptive information


In [0]:

airbnb_v1_clean = airbnb_v1.filter(
    F.col("amenities").isNotNull() &
    F.col("details").isNotNull() &
    (F.col("property_id").isNotNull() | F.col("url").isNotNull())
)


### Derive a basic elevator availability indicator from amenities text


In [0]:

airbnb_v1_f1 = airbnb_v1_clean.withColumn(
    "has_elevator",
    F.lower(F.col("amenities")).contains("elevator")
)




### Extract on-premises parking availability from amenities text


In [0]:
airbnb_v1_f2 = airbnb_v1_f1.withColumn(
    "has_parking_on_premises",
    F.lower(F.col("amenities")).contains("parking on premises")
)



### Identify listings with a private entrance based on amenities text


In [0]:
airbnb_v1_f3 = airbnb_v1_f2.withColumn(
    "has_private_entrance",
    F.lower(F.col("amenities")).contains("private entrance")
)



### Detect explicit allowance of assistance animals from amenities text


In [0]:

airbnb_v1_f4 = airbnb_v1_f3.withColumn(
    "assistance_animals_allowed_explicit",
    F.lower(F.col("amenities")).contains("assistance animals")
)



### Inspect the most frequent cities (used to define the project’s city scope)


In [0]:


top_cities = (
    airbnb_v1_f4
    .where(F.col("location").isNotNull() & (F.length(F.trim("location")) > 0))
    .withColumn("city", F.trim(F.split(F.col("location"), ",").getItem(0)))
    .groupBy("city")
    .agg(F.count("*").alias("listing_count"))
    .orderBy(F.col("listing_count").desc())
    .limit(15)
)
display(top_cities)


### Filter to the final 9 target cities (choosen randomly by us) and persist the v1 city-scope table


In [0]:


final_cities = [
    "Paris",
    "Rome",
    "Dubai",
    "San Francisco",
    "São Paulo",
    "Los Angeles",
    "Rio de Janeiro",
    "New York",
    "Las Vegas"
]

airbnb_v1_9cities = (
    airbnb_v1_f4
    .withColumn("city", F.trim(F.split(F.col("location"), ",").getItem(0)))
    .where(F.col("city").isin(final_cities))
)

airbnb_v1_9cities.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("access4all_airbnb_v1_9cities")
